In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import cv2
import keras
import tensorflow as tf
import PIL

import re

In [81]:
train_data = pd.read_csv('train.csv')
test_data = pd.read_csv('test.csv')

In [82]:
ds = tf.data.Dataset.from_tensor_slices((train_data["example_path"], train_data["label"]))

In [83]:
DATASET_SIZE = train_data.shape[0]

In [84]:
train_size = int(0.8 * DATASET_SIZE)

In [85]:
ds.shuffle(1000)
train = ds.take(train_size)
val = ds.skip(train_size)

In [86]:
def preprocess(image, label):
    img = tf.io.read_file(image)
    img = tf.io.decode_png(img, channels=3,dtype=tf.dtypes.uint16)
    img = tf.cast(img,dtype=tf.float64)
    resized_image = tf.image.resize(img, [332, 332])
    final_image = tf.keras.applications.xception.preprocess_input(resized_image)
    return final_image, label

In [89]:
val

<PrefetchDataset element_spec=(TensorSpec(shape=(332, 332, 3), dtype=tf.float32, name=None), TensorSpec(shape=(), dtype=tf.int64, name=None))>

In [88]:
train = train.map(preprocess).prefetch(tf.data.AUTOTUNE)
val = val.map(preprocess).prefetch(tf.data.AUTOTUNE)

In [90]:
data_augmentation = keras.Sequential(
  [
    tf.keras.layers.RandomFlip("horizontal",
                      input_shape=(332,
                                  332,
                                  3)),
    tf.keras.layers.RandomRotation(0.1),
    tf.keras.layers.RandomZoom(0.1),
  ]
)


In [91]:
def generate_images(df):
    df_batch_1 = df.batch(1, drop_remainder=True).prefetch(tf.data.AUTOTUNE)
    for image, label in df_batch_1:
        if(label == 1):
            conv_im = data_augmentation(image)
            plt.subplot(1, 2, 1)
            plt.imshow(image[0].numpy().astype("uint8"))
            plt.subplot(1, 2, 2)
            plt.imshow(conv_im[0].numpy().astype("uint8"))
            plt.show()
            ds = tf.data.Dataset.from_tensors((conv_im[0], label))
            df = df.concatenate(ds)
    return df

In [54]:
val

<PrefetchDataset element_spec=(TensorSpec(shape=(None, None, 3), dtype=tf.float64, name=None), TensorSpec(shape=(), dtype=tf.int64, name=None))>

In [ ]:
#train = generate_images(train)
val = generate_images(val)

In [242]:
batch_size = 32
train = train.batch(batch_size, drop_remainder=True).prefetch(tf.data.AUTOTUNE)
val = val.batch(batch_size, drop_remainder=True).prefetch(tf.data.AUTOTUNE)

In [ ]:
for image, label in val.batch(1, drop_remainder=True).prefetch(tf.data.AUTOTUNE):
    plt.imshow(image[0].numpy().astype("uint8"))
    plt.show()

In [254]:
len(list(val))

387

In [ ]:
plt.figure(figsize=(10, 10))
for images, _ in train.take(1):
    for i in range(9):
        augmented_images = data_augmentation(images)
        ax = plt.subplot(3, 3, i + 1)
        plt.imshow(augmented_images[0].numpy().astype("uint8"))
        plt.axis("off")